In [ ]:
!pip install nb_black
%load_ext nb_black

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

for dirname, _, filenames in os.walk("/kaggle/input"):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv(
    "/kaggle/input/amazon-top-50-bestselling-books-2009-2019/bestsellers with categories.csv"
)

In [ ]:
df.info()

In [ ]:
y = df["User Rating"]
X = df.drop(["User Rating", "Name"], axis=1)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import lightgbm as lgb

In [ ]:
columns = X.select_dtypes("object").columns
X[columns] = X[columns].astype("category")

In [ ]:
Xt, Xv, yt, yv = train_test_split(X, y, random_state=0)
dt = lgb.Dataset(Xt, yt)
dv = lgb.Dataset(Xv, yv)

In [ ]:
import optuna


def objective(trial):
    metric = "rmse"
    param = {
        "objective": "regression",
        "metric": metric,
        "verbosity": -1,
        "boosting_type": "gbdt",
        "eta": trial.suggest_float("eta", 1e-4, 1, log=False),
    }

    model = lgb.train(
        param,
        dt,
        num_boost_round=10000,
        valid_sets=[dt, dv],
        valid_names=["training", "valid"],
        early_stopping_rounds=50,
        verbose_eval=False,
    )
    return model.best_score["valid"][metric]


study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=100)

print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

best_eta = trial.params["eta"]

In [ ]:
import lightgbm as lgb

In [ ]:
model = lgb.train(
    {"objective": "regression", "metric": "rmse", "eta": best_eta},
    dt,
    num_boost_round=10000,
    valid_sets=[dt, dv],
    valid_names=["training", "valid"],
    early_stopping_rounds=50,
    verbose_eval=10,
)

In [ ]:
lgb.plot_importance(model, grid=False)

In [ ]:
corr = Xt.corr(method="kendall")
corr = corr.stack()
corr = corr.loc[
    [tup for tup in corr.index if tup[0] != tup[1]]
]  # remove pairs of itself
threshold = 0.75
high_corr = corr[(abs(corr) > threshold)]
abs_high_corr = abs(high_corr)[::2]
pairs = abs_high_corr.sort_values(ascending=False).index.to_list()
print(f"Correlated features: {pairs if len(pairs) > 0 else None}")

In [ ]:
sorted_features = [
    feature
    for _, feature in sorted(
        zip(model.feature_importance(), model.feature_name()), reverse=False
    )
]

metric = "rmse"
best_score = model.best_score["valid"][metric]
print(f"starting score: {best_score:.4f}")
unimportant_features = []
for feature in sorted_features:
    unimportant_features.append(feature)
    X_train, X_valid, y_train, y_valid = train_test_split(
        X.drop(unimportant_features, axis=1), y, random_state=0,
    )
    dt = lgb.Dataset(X_train, y_train)
    dv = lgb.Dataset(X_valid, y_valid)
    drop_model = lgb.train(
        {"objective": "regression", "metric": metric, "eta": best_eta},
        dt,
        valid_sets=[dt, dv],
        valid_names=["training", "valid"],
        num_boost_round=10000,
        early_stopping_rounds=50,
        verbose_eval=False,
    )
    score = drop_model.best_score["valid"][metric]
    if score > best_score:
        del unimportant_features[-1]  # remove from drop list
        print(f"Dropping {feature} worsened score to {score:.4f}.")
        break
    else:
        best_score = score
print(f"ending score: {best_score:.4f}")
print(
    f"dropped features: {unimportant_features if len(unimportant_features) > 0 else None}"
)

In [ ]:
X = X.drop(unimportant_features, axis=1)
Xt, Xv, yt, yv = train_test_split(X, y, random_state=0)
dt = lgb.Dataset(Xt, yt)
dv = lgb.Dataset(Xv, yv)

In [ ]:
model = lgb.train(
    {"objective": "regression", "metric": "rmse", "eta": best_eta},
    dt,
    num_boost_round=10000,
    valid_sets=[dt, dv],
    valid_names=["training", "valid"],
    early_stopping_rounds=50,
    verbose_eval=10,
)

In [ ]:
import optuna.integration.lightgbm as lgb

metric = "rmse"

dt = lgb.Dataset(Xt, yt)
dv = lgb.Dataset(Xv, yv)

params = {
    "objective": "regression",
    "metric": metric,
    "verbosity": -1,
    "boosting_type": "gbdt",
    "eta": best_eta,
}

model = lgb.train(
    params,
    dt,
    valid_sets=[dt, dv],
    valid_names=["training", "valid"],
    num_boost_round=10000,
    verbose_eval=False,
    early_stopping_rounds=50,
)

score = model.best_score["valid"][metric]

best_params = model.params
print("Best params:", best_params)
print(f"  {metric} = {score}")
print("  Params: ")
for key, value in best_params.items():
    print("    {}: {}".format(key, value))
best_params['eta'] = best_eta

In [ ]:
import lightgbm as lgb

In [ ]:
model = lgb.train(
    best_params,
    dt,
    num_boost_round=10000,
    valid_sets=[dt, dv],
    valid_names=["training", "valid"],
    early_stopping_rounds=50,
    verbose_eval=10,
)

In [ ]:
lgb.plot_importance(model, grid=False)

In [ ]:
import shap

explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X)
sorted_features = [
    feature
    for _, feature in sorted(
        zip(model.feature_importance(), dt.feature_name), reverse=True
    )
]
# to make sense of it all
for name in sorted_features:
    shap.dependence_plot(name, shap_values, X)

In [ ]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [ ]:
print(f"R2: {r2_score(yv, model.predict(Xv, num_iteration=model.best_iteration)):.3f}")
print(
    f"MAE: {mean_absolute_error(yv, model.predict(Xv, num_iteration=model.best_iteration)):.3f}"
)
print(
    f"RMSE: {np.sqrt(mean_squared_error(yv, model.predict(Xv, num_iteration=model.best_iteration))):.3f}"
)